In [39]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

print('Success')

Success


In [35]:
urls = ['https://github.com/theatlantic/django-cropduster', 'https://github.com/texastribune/data-visuals-create']
github = []
for url in urls:
    response = get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    body = soup.find('article', class_='markdown-body entry-content container-lg').text
    code = soup.find('a', class_='d-inline-flex flex-items-center flex-nowrap link-gray no-underline text-small mr-3').text
    dicob = {'body':body, 'top_code':code}
    github.append(dicob)
github

[{'body': 'django-cropduster\n\n\ndjango-cropduster is a project that makes a form field available that\nuses the Jcrop jQuery plugin. It is a drop-in\nreplacement for django\'s ImageField and allows users to generate multiple crops\nfrom images, using predefined sizes and aspect ratios. django-cropduster\nwas created by developers at The Atlantic. It\nis compatible with python 2.7 and 3.4, and Django versions 1.4 - 1.8.\n\nInstallation\nConfiguration\nDocumentation & Examples\nLicense\n\nInstallation\nThe recommended way to install django-cropduster is from PyPI:\n    pip install django-cropduster\n\nAlternatively, one can install a development copy of django-cropduster from\nsource:\n    pip install -e git+git://github.com/theatlantic/django-cropduster.git#egg=django-cropduster\n\nIf the source is already checked out, use setuptools:\n    python setup.py develop\n\nConfiguration\nTo enable django-cropduster, "cropduster" must be added to INSTALLED_APPS\nin settings.py and you must in

In [46]:
df = pd.DataFrame(github)
df

,body,top_code
0,django-cropduster\n\n\ndjango-cropduster is a ...,\n\nPython\n50.3%\n
1,\n data-visuals-create\n\n\n\n\n\n\nA tool fo...,\n\nJavaScript\n61.3%\n


In [43]:
def basic_clean(somestring):
    basic = somestring.lower()
    basic = unicodedata.normalize('NFKD', basic).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    basic = re.sub(r"[^a-z0-9'\s]", '', basic)
    return basic


def tokenize(somestring):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    tokened = tokenizer.tokenize(somestring, return_str=True)    
    return tokened


def stem(somestring):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in somestring.split()]
    article_stemmed = ' '.join(stems)
    return article_stemmed

def lemmatize(somestring):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in somestring.split()]
    article_lemmatized = ' '.join(lemmas)
    return article_lemmatized

def remove_stopwords(somestring, list_to_remove=[]):
    stopword_list = stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')
    somestring = somestring.lower()
    words = somestring.split()
    filtered_words = [w for w in words if w not in stopword_list]
    article_without_stopwords = ' '.join(filtered_words)
    return article_without_stopwords


In [51]:
df['clean'] = df['body'].apply(basic_clean).apply(tokenize).apply(remove_stopwords)
df['top_code_clean'] = df['top_code'].apply(basic_clean).apply(tokenize).apply(remove_stopwords)
df[['top_code_clean', 'percentage']] = df['top_code_clean'].str.split(' ', expand=True)
df

,body,top_code,clean,top_code_clean,percentage
0,django-cropduster\n\n\ndjango-cropduster is a ...,\n\nPython\n50.3%\n,djangocropduster djangocropduster project make...,python,503
1,\n data-visuals-create\n\n\n\n\n\n\nA tool fo...,\n\nJavaScript\n61.3%\n,datavisualscreate tool generating scaffolding ...,javascript,613


,body,top_code,clean,top_code_clean
0,django-cropduster\n\n\ndjango-cropduster is a ...,\n\nPython\n50.3%\n,djangocropduster djangocropduster project make...,python 503
1,\n data-visuals-create\n\n\n\n\n\n\nA tool fo...,\n\nJavaScript\n61.3%\n,datavisualscreate tool generating scaffolding ...,javascript 613
